In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

import requests
from bs4 import BeautifulSoup
import time
import re
import json
import string
import pandas as pd
import csv
import math
import os
import threading

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

import requests
from bs4 import BeautifulSoup
import time
import re
import json
import string
import pandas as pd
import csv
import math
import os
import threading

https://www.thestar.com.my/search?pgno=2&q=vaccine&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31

https://www.thestar.com.my/search/?q=vaccine&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31

In [ ]:
source = requests.get("https://www.thestar.com.my/search/?q=vaccine&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31").text
soup = BeautifulSoup(source, 'lxml')
print(soup.prettify())

In [ ]:
topics = soup.find_all(class_="row list-listing")

csv_file = open('csv_scrape.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headline', 'date', 'time', 'link'])


for topic in topics:
  headline = topic.div.h2.a.text
  print(headline)
  time_meta = topic.div.span.text
  date_posted = time_meta.split('|')
  date_posted = [item.strip() for item in date_posted]
  reference = topic.div.h2.a.get('href')
  print(date_posted)
  print(reference)
  csv_writer.writerow([headline, date_posted[0], date_posted[1], reference])

csv_file.close()

In [ ]:
csv_file = open('csv_scrape.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headline', 'date', 'time', 'link'])

target = 'vaccine'

for page in range(335):
  print(f'Crawling page {page}')
  url = f'https://www.thestar.com.my/search?pgno={page}&q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
  source = requests.get(url).text
  soup = BeautifulSoup(source, 'lxml')

  topics = soup.find_all(class_="row list-listing")
  for topic in topics:
    headline = topic.div.h2.a.text
    time_meta = topic.div.span.text
    date_posted = time_meta.split('|')
    date_posted = [item.strip() for item in date_posted]
    reference = topic.div.h2.a.get('href')
    csv_writer.writerow([headline, date_posted[0], date_posted[1], reference])

  time.sleep(1)

csv_file.close()

In [ ]:
def crawler_The_Star(targets):

  if isinstance(targets, list):
    pass
  else:
    targets = [targets]

  for target in targets:
    url = f'https://www.thestar.com.my/search?q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    regex = r"a*[0-9]+"
    page_navs = soup.find_all(id="slcontent_0_ThisPageRowCounts")
    for page_nav in page_navs:
      page_list = page_nav.text
      page_iter = re.finditer(regex, page_list)
      for match in page_iter:
        largest_query = match.group(0)
    largest_page = math.ceil(float(largest_query)/30)
    print(f'The total number of query result for the target word {target} is {largest_query}, with {largest_page} pages.')

    csv_file = open(f'{target}.csv', 'w')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['headline', 'date', 'time', 'link'])

    for page in range(1, largest_page):
      print(f'Crawling page {page} of target {target}')
      url = f'https://www.thestar.com.my/search?pgno={page}&q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
      source = requests.get(url).text
      soup = BeautifulSoup(source, 'lxml')

      topics = soup.find_all(class_="row list-listing")
      for topic in topics:
        headline = topic.div.h2.a.text
        time_meta = topic.div.span.text
        date_posted = time_meta.split('|')
        date_posted = [item.strip() for item in date_posted]
        reference = topic.div.h2.a.get('href')
        csv_writer.writerow([headline, date_posted[0], date_posted[1], reference])

      time.sleep(0.5)

    csv_file.close()

In [ ]:
crawler_The_Star('vaccine')

In [ ]:
content_url = 'https://www.thestar.com.my/opinion/letters/2020/01/13/engaging-with-anti-vaxxers'
content_source = requests.get(content_url).text
content_soup = BeautifulSoup(content_source, 'lxml')

content_topics = content_soup.find_all('p')
news_body = []
##print(content_topics)

for content_topic in content_topics:
    content_body = content_topic.text
    #print(content_body)
    news_body.append(content_body)
del news_body[-8:]
news_content = "\n".join(news_body).strip()


In [ ]:
class The_Star_Crawler:
    def __init__(self, targets, sleeptime, verbose):
        if isinstance(targets, list):
            self.targets = targets
        else:
            self.targets = [targets]
        self.sleeptime = sleeptime
        self.verbose = verbose

    def obtainPageNumber(self, target):
        url = f'https://www.thestar.com.my/search?q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        regex = r"a*[0-9]+"
        page_navs = soup.find_all(id="slcontent_0_ThisPageRowCounts")
        for page_nav in page_navs:
            page_list = page_nav.text
            page_iter = re.finditer(regex, page_list)
            for match in page_iter:
                largest_query = match.group(0)
        largest_page = math.ceil(float(largest_query)/30)
        print(f'The total number of query result for the target word {target} is {largest_query}, with {largest_page} pages.')

        return largest_page

    def crawl(self, largest_page, target, sleeptime):
        csv_file = open(f'{target}.csv', 'w')
        csv_writer = csv.writer(csv_file)
        if (self.verbose):
            csv_writer.writerow(['headline', 'date', 'time', 'link', 'content'])
        else:
            csv_writer.writerow(['headline', 'date', 'time', 'link'])

        for page in range(1, largest_page):
            print(f'Crawling page {page} of target {target}')
            url = f'https://www.thestar.com.my/search?pgno={page}&q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
            source = requests.get(url).text
            soup = BeautifulSoup(source, 'lxml')

            topics = soup.find_all(class_="row list-listing")

            for topic in topics:
                headline = topic.div.h2.a.text
                time_meta = topic.div.span.text
                date_posted = time_meta.split('|')
                date_posted = [item.strip() for item in date_posted]
                reference = topic.div.h2.a.get('href')

                if (self.verbose):
                    content_url = reference
                    content_source = requests.get(content_url).text
                    content_soup = BeautifulSoup(content_source, 'lxml')

                    content_topics = content_soup.find_all('p')
                    news_body = []
                    ##print(content_topics)

                    for content_topic in content_topics:
                        content_body = content_topic.text
                        #print(content_body)
                        news_body.append(content_body)
                    del news_body[-8:]
                    news_content = "\n".join(news_body).strip()

                    csv_writer.writerow([headline, date_posted[0], date_posted[1], reference, news_content])
                else:
                    csv_writer.writerow([headline, date_posted[0], date_posted[1], reference])

            time.sleep(sleeptime)

        csv_file.close()

    def run(self):
        for target in self.targets:
            pageNo = self.obtainPageNumber(target)
            self.crawl(pageNo, target, self.sleeptime)


In [ ]:
if __name__ == '__main__':
    targets = ['antigen', 'misinformation', 'covidnow', 'deaths', 'dg', 'noor', 'hisham', 'vaccinated', 'unvaccinated', 'vaccination', 'anti-vaxxers', 'vaccine hesitancy', 'mandate', 'pfizer', 'moderna', 'sinovac', 'az', 'astrazeneca', 'sinopharm', 'mrna', 'ivermectin', 'hydroxychloroquine', 'khairy', 'mco', 'emco', 'cmco', 'rmco', 'covid-19', 'moh', 'distancing', 'mask', 'masking']
    web_crawler = The_Star_Crawler(targets,0.1, False)
    web_crawler.run()

https://www.freemalaysiatoday.com/page/1/?s=vaccine

In [ ]:
!pip install fake_useragent
from fake_useragent import UserAgent

In [ ]:
url = f'https://www.freemalaysiatoday.com/?s=vaccine'
ua = UserAgent()
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
source = requests.get(url, headers = headers)
print(source.status_code)
print(source.json())
soup = BeautifulSoup(source.text, 'lxml')
page_navs = soup.find_all('a')
print(page_navs)

In [ ]:
class FMT_Crawler:
    def __init__(self, targets, sleeptime):
        if isinstance(targets, list):
            self.targets = targets
        else:
            self.targets = [targets]
        self.sleeptime = sleeptime

    def obtainPageNumber(self, target):
        url = f'https://www.freemalaysiatoday.com/?s={target}'
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        page_navs = soup.find_all(class_="last")
        print(page_navs)
        for page_nav in page_navs:
            page_list = page_nav.text
            print(page_list)
        largest_page = int(page_list)
        print(f'The total number of query page for the target word {target} is {largest_page}.')

        return largest_page

    def crawl(self, largest_page, target, sleeptime):
        csv_file = open(f'{target}.csv', 'w')
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['headline', 'date', 'time', 'link', 'content'])

        for page in range(1, largest_page):
            print(f'Crawling page {page} of target {target}')
            url = f'https://www.freemalaysiatoday.com/page/{page}/?s={target}'
            source = requests.get(url).text
            soup = BeautifulSoup(source, 'lxml')

            topics = soup.find_all(class_="td_module_10 td_module_wrap td-animation-stack")

            for topic in topics:
                headline = topic.div.h2.a.text
                time_meta = topic.div.span.text
                date_posted = time_meta.split('|')
                date_posted = [item.strip() for item in date_posted]
                reference = topic.div.h2.a.get('href')

                content_url = reference
                content_source = requests.get(content_url).text
                content_soup = BeautifulSoup(content_source, 'lxml')

                content_topics = content_soup.find_all('p')
                news_body = []
                ##print(content_topics)

                for content_topic in content_topics:
                    content_body = content_topic.text
                    #print(content_body)
                    news_body.append(content_body)
                del news_body[-8:]
                news_content = "\n".join(news_body).strip()

                csv_writer.writerow([headline, date_posted[0], date_posted[1], reference, news_content])

            time.sleep(sleeptime)

        csv_file.close()

    def run(self):
        for target in self.targets:
            pageNo = self.obtainPageNumber(target)
            self.crawl(pageNo, target, self.sleeptime)

In [ ]:
if __name__ == '__main__':
    web_crawler = FMT_Crawler('vaccine',0.5)
    web_crawler.run()

In [ ]:
['ventilator', 'pcr', 'testing', 'antigen', 'misinformation', 'covidnow', 'deaths', 'dg', 'noor', 'hisham', 'vaccinated', 'unvaccinated', 'vaccination', 'anti-vaxxers', 'vaccine hesitancy', 'mandate', 'pfizer', 'sinovac', 'az', 'astrazeneca', 'sinopharm', 'mrna', 'ivermectin', 'hydroxychloroquine', 'khairy', 'mco', 'emco', 'cmco', 'rmco', 'covid-19', 'moh', 'distancing', 'mask', 'masking']

In [ ]:
response = requests.get('https://www.sinchew.com.my/coronavirus/')
response.status_code

403

In [ ]:
def create_project_directory(directory):
    if not os.path.exists(directory):
        print(f'Creating project {directory}')
        os.makedirs(directory)

def create_data_files(project_name, base_url):
    queue = project_name + '/queue.txt'
    crawled = project_name + '/crawled.txt'
    if not os.path.isfile(queue):
        write_file(queue, base_url)
    if not os.path.isfile(crawled):
        write_file(crawled, '')

def write_file(path, data):
    with open(path, 'w') as f:
        f.write(data)
        f.close()

# Add data onto an existing file
def append_to_file(path, data):
    with open(path, 'a') as file:
        file.write(data + '\n')

# Delete the contents of a file
def delete_file_contents(path):
    with open(path, 'w'):
        pass

# Read a file and convert each line to set items
def file_to_set(file_name):
    results = set()
    with open(file_name, 'rt') as f:
        for line in f:
            results.add(line.replace('\n', ''))
    return results


# Iterate through a set, each item will be a line in a file
def set_to_file(links, file_name):
    with open( file_name,"w") as f:
        for l in sorted(links):
            f.write(l+"\n")

In [ ]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,))
    x.start()
    x = threading.Thread(target=thread_function, args=(2,))
    x.start()
    logging.info("Main    : before running thread")
    logging.info("Main    : wait for the thread to finish")
    # x.join()
    logging.info("Main    : all done")

00:58:48: Main    : before creating thread
00:58:48: Thread 1: starting
00:58:48: Thread 2: starting
00:58:48: Main    : before running thread
00:58:48: Main    : wait for the thread to finish
00:58:48: Main    : all done


In [ ]:
class The_Star_Crawler:
    def __init__(self, targets, sleeptime, directory, verbose):
        if isinstance(targets, list):
            self.targets = targets
        else:
            self.targets = [targets]
        self.sleeptime = sleeptime
        self.verbose = verbose
        self.directory = directory

    def obtainPageNumber(self, target):
        url = f'https://www.thestar.com.my/search?q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        regex = r"a*[0-9]+"
        page_navs = soup.find_all(id="slcontent_0_ThisPageRowCounts")
        for page_nav in page_navs:
            page_list = page_nav.text
            page_iter = re.finditer(regex, page_list)
            for match in page_iter:
                largest_query = match.group(0)
        largest_page = math.ceil(float(largest_query)/30)
        print(f'The total number of query result for the target word {target} is {largest_query}, with {largest_page} pages.')

        return largest_page

    def create_project(self, directory):
        if not os.path.exists(directory):
            print(f'Creating project {directory}')
            os.makedirs(directory)

    def crawl(self, largest_page, target, sleeptime):
        csv_file = open(f'{self.directory}/{target}.csv', 'w')
        csv_writer = csv.writer(csv_file)
        if (self.verbose):
            csv_writer.writerow(['headline', 'date', 'time', 'link', 'content'])
        else:
            csv_writer.writerow(['headline', 'date', 'time', 'link'])

        for page in range(1, largest_page):
            print(f'Crawling page {page} of target {target}')
            url = f'https://www.thestar.com.my/search?pgno={page}&q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
            time.sleep(0.015)
            source = requests.get(url).text
            soup = BeautifulSoup(source, 'lxml')

            topics = soup.find_all(class_="row list-listing")

            for topic in topics:
                headline = topic.div.h2.a.text
                time_meta = topic.div.span.text
                date_posted = time_meta.split('|')
                date_posted = [item.strip() for item in date_posted]
                reference = topic.div.h2.a.get('href')

                if (self.verbose):
                    content_url = reference
                    content_source = requests.get(content_url).text
                    content_soup = BeautifulSoup(content_source, 'lxml')

                    content_topics = content_soup.find_all('p')
                    news_body = []
                    ##print(content_topics)

                    for content_topic in content_topics:
                        content_body = content_topic.text
                        #print(content_body)
                        news_body.append(content_body)
                    del news_body[-8:]
                    news_content = "\n".join(news_body).strip()

                    csv_writer.writerow([headline, date_posted[0], date_posted[1], reference, news_content])
                else:
                    csv_writer.writerow([headline, date_posted[0], date_posted[1], reference])

            time.sleep(sleeptime)

        csv_file.close()


    def run(self, target):
        pageNo = self.obtainPageNumber(target)
        self.crawl(pageNo, target, self.sleeptime)

    def multirun(self):
        self.create_project(self.directory)

        for index, target in enumerate(self.targets):
            logging.info("Main    : create and start thread %d.", index)
            t = threading.Thread(target=self.run, args = (target, ))
            t.start()
        t.join()

        !zip -r f'/content/{directory}{('Summary','')[verbose]}.zip /content/{directory}'
        from google.colab import files
        files.download(f"/content/{directory}{('Summary','')[verbose]}.zip")

In [ ]:
if __name__ == '__main__':
    targets = ['virus', 'ventilator', 'pcr', 'testing', 'antigen', 'misinformation', 'covidnow', 'deaths', 'dg', 'noor', 'hisham', 'vaccinated', 'unvaccinated', 'vaccination', 'anti-vaxxers', 'vaccine hesitancy', 'mandate', 'pfizer', 'sinovac', 'az', 'astrazeneca', 'sinopharm', 'mrna', 'ivermectin', 'hydroxychloroquine', 'khairy', 'mco', 'emco', 'cmco', 'rmco', 'covid-19', 'moh', 'distancing', 'mask', 'masking', 'fines']
    web_crawler = The_Star_Crawler(targets, 0.1, 'TheStar', False)
    web_crawler.multirun()